# Testing Performance of AerialWaste Model

In [1]:
import sys

sys.path.append("/kaggle/input/aerial-waste-model")
sys.path.append("/kaggle/input/aerial-waste-save-cams/aerialwaste-model")

In [2]:
import json
import os
import torch
import time
import gc
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils.image_processor import ImageProcessor

In [3]:
# # Step 1: Set up paths
# TEST_JSON_PATH = r"/kaggle/input/aerialwaste-dataset/12607190/testing.json"
# TEST_IMAGE_FOLDER = r"/kaggle/input/aerialwaste-dataset/12607190/images/"
# STATE_DICT_PATH = r"/kaggle/input/weights-aerial-waste-model/weights/checkpoint.pth"

# # Step 2: Define categories to detect suspicious sites
# CATS = ["suspicious_site"]
# model = 'architecture.resnet50_fpn'

# # Step 3: Load the Image Processor
# ip = ImageProcessor(CATS, STATE_DICT_PATH, model=model)
# print("Model loaded successfully.")

# # Step 4: Check if GPU is available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using device: {device}")

# # Step 5: Load test dataset
# with open(TEST_JSON_PATH, 'r') as f:
#     test_data = json.load(f)

# # Step 6: Extract test images
# test_images = test_data.get("images", [])
# print(f"Found {len(test_images)} test images.")

# # Step 7: Run predictions in Batches
# batch_size = 50  # Adjust if OOM persists
# num_batches = (len(test_images) + batch_size - 1) // batch_size

# output_json_path = "/kaggle/working/test_predictions.json"

# for i in tqdm(range(0, len(test_images), batch_size), desc="Processing Batches"):
#     batch = test_images[i:i+batch_size]

#     results = []
#     for image_data in tqdm(batch, desc=f"Processing Batch {i//batch_size + 1}/{num_batches}", leave=False):
#         img_path = os.path.join(TEST_IMAGE_FOLDER, image_data["file_name"])
#         if not os.path.exists(img_path):
#             print(f"Warning: Image {img_path} not found. Skipping.")
#             continue

#         start_time = time.time()

#         with torch.no_grad():  # Fix 1: Prevents storing unnecessary computation graphs
#             iw = ip.execute_cams_pred(img_path)

#         elapsed_time = time.time() - start_time
#         print(f"Processed {image_data['file_name']} in {elapsed_time:.2f} sec")

#         results.append({
#             "image_id": image_data["id"],
#             "file_name": image_data["file_name"],
#             "classification_scores": float(iw.classification_scores[0]),
#             "predicted_categories": list(iw.predicted_categories.keys())[0] if iw.predicted_categories else 0
#         })

#     # Fix 5: Save results after every batch (instead of storing all in memory)
#     with open(output_json_path, 'a') as f:
#         for result in results:
#             json.dump(result, f)
#             f.write("\n")

#     # Fix 2: Free GPU and CPU memory after each batch
#     torch.cuda.empty_cache()
#     gc.collect()

# print(f"✅ Predictions saved at {output_json_path}")

In [4]:
# # Step 1: Create an empty list to store data
# data = []

# # Step 2: Loop through predictions and add necessary columns
# for image_data, prediction in zip(test_images, results):
#     img_id = image_data["id"]
#     file_name = image_data["file_name"]
#     true_class = image_data["is_candidate_location"]  # Ground truth
#     classification_score = prediction["classification_scores"]  # Extract score
#     predicted_category = prediction["predicted_categories"]  # Extract category

#     # Assign predicted class: 1 if 'suspicious_site', else 0
#     predicted_class = 1 if predicted_category == "suspicious_site" else 0

#     # Append to data list
#     data.append({
#         "image_id": img_id,
#         "file_name": file_name,
#         "classification_score": classification_score,
#         "predicted_class": predicted_class,
#         "true_class": true_class
#     })

# # Step 3: Convert to DataFrame
# df = pd.DataFrame(data)

# # Step 4: Save to CSV (optional)
# df.to_csv("/kaggle/working/model_predictions.csv", index=False)
# df

In [5]:
# incorrect_images = df[df["predicted_class"] != df["true_class"]]
# print(f"Found {len(incorrect_images)} incorrectly classified images.")

# for _, row in incorrect_images.iterrows():
#     img_path = os.path.join(TEST_IMAGE_FOLDER, row["file_name"])
#     print(f"Analyzing {row['file_name']}...")
#     iw = ip.execute_cams_pred(img_path)
#     print(iw.classification_scores, iw.predicted_categories)
#     iw.show_global_cams()


In [6]:
# from sklearn.metrics import classification_report, accuracy_score, f1_score

# # Extract true and predicted labels
# y_true = df["true_class"]
# y_pred = df["predicted_class"]

# # Compute accuracy
# accuracy = accuracy_score(y_true, y_pred)

# # Compute F1-score
# f1 = f1_score(y_true, y_pred)

# # Generate classification report
# report = classification_report(y_true, y_pred, target_names=["Negative (0)", "Positive (1)"])

# # Display results
# print(f"✅ Accuracy: {accuracy:.4f}")
# print(f"✅ F1 Score: {f1:.4f}")
# print("\nClassification Report:\n", report)

In [7]:
# # Compute classification report as a dictionary
# report_dict = classification_report(y_true, y_pred, target_names=["Negative (0)", "Positive (1)"], output_dict=True)

# # Convert to DataFrame
# report_df = pd.DataFrame(report_dict).transpose()

# # Save to CSV
# report_csv_path = "/kaggle/working/classification_report.csv"
# report_df.to_csv(report_csv_path, index=True)
# report_df

# With uploaded predictions

In [8]:
# Step 1: Set up paths
TEST_JSON_PATH = r"/kaggle/input/aerialwaste-dataset/12607190/testing.json"
TRAIN_JSON_PATH = r"/kaggle/input/aerialwaste-dataset/12607190/training.json"
TEST_IMAGE_FOLDER = r"/kaggle/input/aerialwaste-dataset/12607190/images/"
STATE_DICT_PATH = r"/kaggle/input/weights-aerial-waste-model/weights/checkpoint.pth"

# Step 2: Define categories to detect suspicious sites
CATS = ["suspicious_site"]
model = 'architecture.resnet50_fpn'

# Step 3: Load the Image Processor
ip = ImageProcessor(CATS, STATE_DICT_PATH, model=model)
print("Model loaded successfully.")

# Step 4: Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Step 5.1: Load test dataset
with open(TEST_JSON_PATH, 'r') as f:
    test_data = json.load(f)

# Step 5.2: Load train dataset
with open(TRAIN_JSON_PATH, 'r') as f:
    train_data = json.load(f)

# Step 6.1: Extract test images
test_images = test_data.get("images", [])
print(f"Found {len(test_images)} test images.")

# Step 6.2: Extract test images
train_images = train_data.get("images", [])
print(f"Found {len(train_images)} train images.")

Model loaded successfully.
Using device: cuda
Found 2607 test images.
Found 9096 train images.


In [9]:
with open("/kaggle/input/test-predictions/test_predictions.json", "r") as file:
    dic = [json.loads(line) for line in file]

df = pd.DataFrame(dic)
df["predicted_class"] = df["classification_scores"].apply(
    lambda x: 1 if x >= 0.44 else 0
)
df.drop(columns=["predicted_categories"], inplace=True)
df

,image_id,file_name,classification_scores,predicted_class
0,13,13.png,6.776331e-01,1
1,15,15.png,2.463700e-01,0
2,16,16.png,6.776331e-01,1
3,17,17.png,6.776331e-01,1
4,21,21.png,6.776331e-01,1
...,...,...,...,...
2602,11696,11696.png,1.301220e-07,0
2603,11697,11697.png,6.064392e-02,0
2604,11700,11700.png,3.604656e-07,0
2605,11701,11701.png,2.979757e-07,0


In [10]:
# Convert list of dictionaries to DataFrame
true_class_df = pd.DataFrame(test_images)

# Rename column 'id' to 'image_id' to match df
true_class_df.rename(columns={"id": "image_id", "is_candidate_location": "true_class"}, inplace=True)

# Merge the two DataFrames on 'image_id'
df = df.merge(true_class_df[["image_id", "true_class", "severity", "evidence", "site_type", "categories"]], on="image_id", how="left")

In [11]:
categories = test_data["categories"]

In [12]:
# Create a dictionary mapping ID -> Name
category_mapping = {cat["id"]: cat["name"] for cat in categories}

# Function to replace IDs with Names
def replace_category_ids(id_list):
    return [category_mapping.get(cat_id, "Unknown") for cat_id in id_list]

# Apply transformation
df["categories"] = df["categories"].apply(replace_category_ids)
df

,image_id,file_name,classification_scores,predicted_class,true_class,severity,evidence,site_type,categories
0,13,13.png,6.776331e-01,1,1,1,1,Production site,[]
1,15,15.png,2.463700e-01,0,0,1,1,Agricultural area/farm area,[]
2,16,16.png,6.776331e-01,1,1,1,2,Non production building,[]
3,17,17.png,6.776331e-01,1,1,2,1,Production site,[]
4,21,21.png,6.776331e-01,1,1,1,1,Agricultural area/farm area,[]
...,...,...,...,...,...,...,...,...,...
2602,11696,11696.png,1.301220e-07,0,0,n/a,n/a,n/a,[]
2603,11697,11697.png,6.064392e-02,0,0,n/a,n/a,n/a,[]
2604,11700,11700.png,3.604656e-07,0,0,n/a,n/a,n/a,[]
2605,11701,11701.png,2.979757e-07,0,0,n/a,n/a,n/a,[]


In [13]:
# from sklearn.metrics import classification_report, accuracy_score, f1_score

# # Extract true and predicted labels
# y_true = df["true_class"]
# y_pred = df["predicted_class"]

# # Compute accuracy
# accuracy = accuracy_score(y_true, y_pred)

# # Compute F1-score
# f1 = f1_score(y_true, y_pred)

# # Generate classification report
# report = classification_report(y_true, y_pred, target_names=["Negative (0)", "Positive (1)"])

# # Display results
# print(f"✅ Accuracy: {accuracy:.4f}")
# print(f"✅ F1 Score: {f1:.4f}")
# print("\nClassification Report:\n", report)

In [14]:
# # False Positives: Predicted 1, but actually 0
# false_pos = df[(df["predicted_class"] == 1) & (df["true_class"] == 0)]
# false_pos.to_csv("/kaggle/working/false_pos.csv")

# # False Negatives: Predicted 0, but actually 1
# false_neg = df[(df["predicted_class"] == 0) & (df["true_class"] == 1)]
# false_neg.to_csv("/kaggle/working/false_neg.csv")

# # Print results
# print(f"Found {len(false_pos + false_neg)} incorrectly classified images.")
# print(f"False Positives: {len(false_pos)}")
# print(f"False Negatives: {len(false_neg)}")

In [15]:
# false_pos_output = "/kaggle/working/false_pos_images"
# false_neg_output = "/kaggle/working/false_neg_images"
# os.makedirs(false_pos_output, exist_ok=True)
# os.makedirs(false_neg_output, exist_ok=True)

# # 🔹 **Process False Positives**
# print("\nProcessing False Positives...\n")
# for _, row in false_pos.iterrows():
#     img_path = os.path.join(TEST_IMAGE_FOLDER, row["file_name"])
#     print(f"Analyzing False Positive: {row['file_name']}...")
#     print(f"Severity: {row['severity']}")
#     print(f"Evidence: {row['evidence']}")
#     print(f"Site type: {row['site_type']}")
#     print(f"Categories: {row['categories']}")

#     # Execute classification analysis
#     iw = ip.execute_cams_pred(img_path)

#     # Manually trigger show_global_cams (which will plot the image)
#     plt.figure()  # Create a new figure
#     iw.show_global_cams()  # This displays the CAM visualization

#     # Save the visualization manually
#     fig = plt.gcf()
#     save_path = os.path.join(false_pos_output, f"{row['image_id']}_cam.png")
#     plt.savefig(save_path, bbox_inches="tight")
#     plt.close(fig)  # Close figure to free memory

#     print(f"Saved image to: {save_path}")

# # 🔹 **Process False Negatives**
# print("\nProcessing False Negatives...\n")
# for _, row in false_neg.iterrows():
#     img_path = os.path.join(TEST_IMAGE_FOLDER, row["file_name"])
#     print(f"Analyzing False Negative: {row['file_name']}...")
#     print(f"Severity: {row['severity']}")
#     print(f"Evidence: {row['evidence']}")
#     print(f"Site type: {row['site_type']}")
#     print(f"Categories: {row['categories']}")

#     # Execute classification analysis
#     iw = ip.execute_cams_pred(img_path)

#     # Manually trigger show_global_cams (which will plot the image)
#     plt.figure()  # Create a new figure
#     iw.show_global_cams()  # This displays the CAM visualization

#     # Save the visualization manually
#     fig = plt.gcf()
#     save_path = os.path.join(false_neg_output, f"{row['image_id']}_cam.png")
#     plt.savefig(save_path, bbox_inches="tight")
#     plt.close(fig)  # Close figure to free memory

#     print(f"Saved image to: {save_path}")

# Inception V3
## Fine-tuning

In [16]:
from torchvision.models import inception_v3, Inception_V3_Weights
import torch.nn as nn

In [17]:
weights = Inception_V3_Weights.IMAGENET1K_V1
model = inception_v3(weights=weights)

# Move model to GPU (if available)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 82.0MB/s] 


In [18]:
# Modify Fully Connected (FC) Layer for Binary Classification
num_ftrs = model.fc.in_features  # Get number of input features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Suspicious (1) or Normal (0)

# Move model to device (again, after modification)
model = model.to(device)

## Fine tune on AerialWaste Dataset

In [19]:
from torchinfo import summary

In [20]:
train = train_images

In [21]:
# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

In [22]:
summary(model)

Layer (type:depth-idx)                   Param #
Inception3                               --
├─BasicConv2d: 1-1                       --
│    └─Conv2d: 2-1                       (864)
│    └─BatchNorm2d: 2-2                  (64)
├─BasicConv2d: 1-2                       --
│    └─Conv2d: 2-3                       (9,216)
│    └─BatchNorm2d: 2-4                  (64)
├─BasicConv2d: 1-3                       --
│    └─Conv2d: 2-5                       (18,432)
│    └─BatchNorm2d: 2-6                  (128)
├─MaxPool2d: 1-4                         --
├─BasicConv2d: 1-5                       --
│    └─Conv2d: 2-7                       (5,120)
│    └─BatchNorm2d: 2-8                  (160)
├─BasicConv2d: 1-6                       --
│    └─Conv2d: 2-9                       (138,240)
│    └─BatchNorm2d: 2-10                 (384)
├─MaxPool2d: 1-7                         --
├─InceptionA: 1-8                        --
│    └─BasicConv2d: 2-11                 --
│    │    └─Conv2d: 3-1         

In [23]:
# Unfreeze only final layers (fc + Mixed_7)
for name, param in model.named_parameters():
    if "fc" in name or "Mixed_7" in name:
        param.requires_grad = True  # Allow training

In [24]:
summary(model)

Layer (type:depth-idx)                   Param #
Inception3                               --
├─BasicConv2d: 1-1                       --
│    └─Conv2d: 2-1                       (864)
│    └─BatchNorm2d: 2-2                  (64)
├─BasicConv2d: 1-2                       --
│    └─Conv2d: 2-3                       (9,216)
│    └─BatchNorm2d: 2-4                  (64)
├─BasicConv2d: 1-3                       --
│    └─Conv2d: 2-5                       (18,432)
│    └─BatchNorm2d: 2-6                  (128)
├─MaxPool2d: 1-4                         --
├─BasicConv2d: 1-5                       --
│    └─Conv2d: 2-7                       (5,120)
│    └─BatchNorm2d: 2-8                  (160)
├─BasicConv2d: 1-6                       --
│    └─Conv2d: 2-9                       (138,240)
│    └─BatchNorm2d: 2-10                 (384)
├─MaxPool2d: 1-7                         --
├─InceptionA: 1-8                        --
│    └─BasicConv2d: 2-11                 --
│    │    └─Conv2d: 3-1         

Define optimizer and loss criterion

In [25]:
import torch.optim as optim

# Use a small learning rate for fine-tuning
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

Load and preprocess train images

In [26]:
from torch.utils.data import Dataset, DataLoader

# Load Pretrained Weights
weights = Inception_V3_Weights.DEFAULT  # Automatically picks best ImageNet weights
preprocess = weights.transforms()  # Get the exact required transformations

In [27]:
from PIL import Image

class LandfillDataset(Dataset):
    def __init__(self, data_list, image_folder, transform=None):
        # Convert dictionary to list if needed
        if isinstance(data_list, dict):
            self.data_list = list(data_list.values())  # Convert to list
        else:
            self.data_list = data_list

        self.image_folder = image_folder
        self.transform = transform  # Use weights.transforms()

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        sample = self.data_list[idx]
        img_path = os.path.join(self.image_folder, sample["file_name"])

        # Open Image
        image = Image.open(img_path).convert("RGB")

        # Apply Pretrained Transformations
        if self.transform:
            image = self.transform(image)

        # Extract label (1 = illegal landfill, 0 = normal site)
        label = torch.tensor(sample["is_candidate_location"], dtype=torch.long)
        return image, label


In [28]:
# Load dataset with weights.transforms()
#### ATTENTION CHANGE train ####
train_dataset = LandfillDataset(data_list=train, image_folder=TEST_IMAGE_FOLDER, transform=preprocess)

# Use DataLoader for batching
train_loader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=2, persistent_workers=True
)


Fine tune (train the last layers) the model on AerialWaste train dataset

In [29]:
from tqdm import tqdm

num_epochs = 10  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    # Use tqdm to add a progress bar
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)  # Returns InceptionOutputs (logits, auxiliary_logits)
        logits = outputs.logits  # Extract only main output for loss calculation

        loss = criterion(logits, labels)  # Compute loss using logits
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Update tqdm progress bar with loss
        progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

    accuracy = 100 * correct / total
    print(f"✅ Epoch [{epoch+1}/{num_epochs}] Completed - Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("🎉 Fine-Tuning Complete!")

Epoch 1/10: 100%|██████████| 285/285 [05:46<00:00,  1.22s/it, acc=78.4, loss=0.527]


✅ Epoch [1/10] Completed - Loss: 129.2426, Accuracy: 78.40%


Epoch 2/10: 100%|██████████| 285/285 [03:42<00:00,  1.28it/s, acc=86.3, loss=0.374]


✅ Epoch [2/10] Completed - Loss: 90.3184, Accuracy: 86.28%


Epoch 3/10: 100%|██████████| 285/285 [03:34<00:00,  1.33it/s, acc=89.4, loss=0.39]


...

✅ Epoch [8/10] Completed - Loss: 16.6324, Accuracy: 98.61%


Epoch 9/10: 100%|██████████| 285/285 [03:38<00:00,  1.31it/s, acc=98.8, loss=0.016]


✅ Epoch [9/10] Completed - Loss: 14.5656, Accuracy: 98.76%


Epoch 10/10: 100%|██████████| 285/285 [03:38<00:00,  1.31it/s, acc=99.2, loss=0.0832]

✅ Epoch [10/10] Completed - Loss: 10.0787, Accuracy: 99.23%
🎉 Fine-Tuning Complete!


## Inference

In [30]:
import torch.nn.functional as F

model.eval().to(device)

# Get Preprocessing Transforms
preprocess = weights.transforms()

# Define Batch Size (Adjust Based on Memory)
batch_size = 50  
output_json_path = "/kaggle/working/test_predictions_inceptionv3.json"

# Ensure Output File is Empty Before Writing
open(output_json_path, 'w').close()

# Process Test Images in Batches
num_batches = (len(test_images) + batch_size - 1) // batch_size  # Total batches

for i in tqdm(range(0, len(test_images), batch_size), desc="Processing Batches"):
    batch = test_images[i:i+batch_size]
    batch_images = []
    batch_ids = []
    batch_filenames = []

    # Load & Preprocess Images
    for image_data in batch:
        img_path = os.path.join(TEST_IMAGE_FOLDER, image_data["file_name"])
        if not os.path.exists(img_path):
            print(f"⚠️ Warning: Image {img_path} not found. Skipping.")
            continue

        img = Image.open(img_path).convert("RGB")
        img_transformed = preprocess(img).unsqueeze(0).to(device)

        batch_images.append(img_transformed)
        batch_ids.append(image_data["id"])
        batch_filenames.append(image_data["file_name"])

    if not batch_images:
        continue  # Skip empty batch

    batch_images = torch.cat(batch_images)  # Stack images into a batch

    # Model Prediction
    with torch.no_grad():
        outputs = model(batch_images)  # Get logits

    probabilities = F.softmax(outputs, dim=1)  # Convert logits to probabilities
    predicted_classes = torch.argmax(probabilities, dim=1).cpu().tolist()  # Get class indices
    predicted_probs = torch.max(probabilities, dim=1).values.cpu().tolist()  # Get max probability

    # Store Results
    results = []
    for img_id, file_name, pred_class, pred_prob in zip(batch_ids, batch_filenames, predicted_classes, predicted_probs):
        results.append({
            "image_id": img_id,
            "file_name": file_name,
            "classification_scores": pred_prob,
            "predicted_categories": pred_class
        })

    # Save Predictions to JSON After Every Batch
    with open(output_json_path, 'a') as f:
        for result in results:
            json.dump(result, f)
            f.write("\n")

    # Free GPU Memory After Each Batch
    torch.cuda.empty_cache()
    gc.collect()

print(f"✅ All Predictions Saved at: {output_json_path}")

Processing Batches: 100%|██████████| 53/53 [03:24<00:00,  3.86s/it]

✅ All Predictions Saved at: /kaggle/working/test_predictions_inceptionv3.json


In [31]:
df.drop(columns=["classification_scores", "predicted_class"], inplace=True)
df

,image_id,file_name,true_class,severity,evidence,site_type,categories
0,13,13.png,1,1,1,Production site,[]
1,15,15.png,0,1,1,Agricultural area/farm area,[]
2,16,16.png,1,1,2,Non production building,[]
3,17,17.png,1,2,1,Production site,[]
4,21,21.png,1,1,1,Agricultural area/farm area,[]
...,...,...,...,...,...,...,...
2602,11696,11696.png,0,n/a,n/a,n/a,[]
2603,11697,11697.png,0,n/a,n/a,n/a,[]
2604,11700,11700.png,0,n/a,n/a,n/a,[]
2605,11701,11701.png,0,n/a,n/a,n/a,[]


In [32]:
with open("/kaggle/working/test_predictions_inceptionv3.json", "r") as file:
    dic = [json.loads(line) for line in file]

df_inceptionv3 = pd.DataFrame(dic)
df_inceptionv3

,image_id,file_name,classification_scores,predicted_categories
0,13,13.png,0.998833,1
1,15,15.png,0.600342,1
2,16,16.png,0.977015,0
3,17,17.png,0.521911,0
4,21,21.png,0.999216,1
...,...,...,...,...
2602,11696,11696.png,0.999790,0
2603,11697,11697.png,0.994052,0
2604,11700,11700.png,0.999316,0
2605,11701,11701.png,0.999675,0


In [33]:
df_inceptionv3 = df_inceptionv3.merge(df, on="image_id", how="left")
df_inceptionv3

,image_id,file_name_x,classification_scores,predicted_categories,file_name_y,true_class,severity,evidence,site_type,categories
0,13,13.png,0.998833,1,13.png,1,1,1,Production site,[]
1,15,15.png,0.600342,1,15.png,0,1,1,Agricultural area/farm area,[]
2,16,16.png,0.977015,0,16.png,1,1,2,Non production building,[]
3,17,17.png,0.521911,0,17.png,1,2,1,Production site,[]
4,21,21.png,0.999216,1,21.png,1,1,1,Agricultural area/farm area,[]
...,...,...,...,...,...,...,...,...,...,...
2602,11696,11696.png,0.999790,0,11696.png,0,n/a,n/a,n/a,[]
2603,11697,11697.png,0.994052,0,11697.png,0,n/a,n/a,n/a,[]
2604,11700,11700.png,0.999316,0,11700.png,0,n/a,n/a,n/a,[]
2605,11701,11701.png,0.999675,0,11701.png,0,n/a,n/a,n/a,[]


In [34]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Extract true and predicted labels
y_true = df_inceptionv3["true_class"]
y_pred = df_inceptionv3["predicted_categories"]

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)

# Compute F1-score
f1 = f1_score(y_true, y_pred)

# Generate classification report
report = classification_report(y_true, y_pred, target_names=["Negative (0)", "Positive (1)"])

# Display results
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ F1 Score: {f1:.4f}")
print("\nClassification Report:\n", report)

✅ Accuracy: 0.8799
✅ F1 Score: 0.8076

Classification Report:
               precision    recall  f1-score   support

Negative (0)       0.89      0.94      0.91      1738
Positive (1)       0.87      0.76      0.81       869

    accuracy                           0.88      2607
   macro avg       0.88      0.85      0.86      2607
weighted avg       0.88      0.88      0.88      2607



In [35]:
# False Positives: Predicted 1, but actually 0
false_pos = df_inceptionv3[(df_inceptionv3["predicted_categories"] == 1) & (df_inceptionv3["true_class"] == 0)]
false_pos.to_csv("/kaggle/working/false_pos_inceptionv3.csv")

# False Negatives: Predicted 0, but actually 1
false_neg = df_inceptionv3[(df_inceptionv3["predicted_categories"] == 0) & (df_inceptionv3["true_class"] == 1)]
false_neg.to_csv("/kaggle/working/false_neg_inceptionv3.csv")

# Print results
print(f"Found {len(false_pos + false_neg)} incorrectly classified images.")
print(f"False Positives: {len(false_pos)}")
print(f"False Negatives: {len(false_neg)}")

Found 313 incorrectly classified images.
False Positives: 101
False Negatives: 212


## Test Inference on Single Image

In [36]:
# import torch.nn.functional as F

# preprocess = weights.transforms()  # Get predefined preprocessing transforms

# # Load an image and apply transformations
# img_path = os.path.join(TEST_IMAGE_FOLDER, "13.png")
# img = Image.open(img_path).convert("RGB")  # Ensure 3 channels (RGB)

# # Apply the transformation pipeline from weights
# img_transformed = preprocess(img).unsqueeze(0)  # Add batch dimension

# # Move to device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# img_transformed = img_transformed.to(device)

# # Pass through the model
# model.eval()
# with torch.no_grad():
#     output = model(img_transformed)

# # Apply Softmax to Convert Logits to Probabilities
# probabilities = F.softmax(output, dim=1)  # Compute softmax across class dimension
# predicted_prob = torch.max(probabilities).item()

# # Get the predicted class
# _, predicted = torch.max(output, 1)
# print(f"Predicted Class: {predicted.item()}")
# print(f"Probability: {predicted_prob:.4f}")

In [37]:
# df.sort_values(by="image_id")